# Homework 2 MLE and Naive Bayes

## MLE

### T1 and OT1

see the picture below



## Simple Bayes Classifier

### T2





In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy import stats
import math
import matplotlib.pyplot as plt

In [ ]:
def plot_bayes_classifier(mu_1, mu_2, vars_1, vars_2, p_w1, p_w2, intersect_x=None):
  x_1 = np.linspace(mu_1 - 4*vars_1, mu_1 + 4*vars_1, 10000)
  y_1 = stats.norm.pdf(x_1, mu_1, vars_1) * p_w1
  x_2 = np.linspace(mu_2 - 4*vars_2, mu_2 + 4*vars_2, 10000)
  y_2 = stats.norm.pdf(x_2, mu_2, vars_2) * p_w2

  if(intersect_x is None):
    intersect_x = (2*vars_1**2*math.log(p_w2) - 2*vars_2**2*math.log(p_w1) + mu_1**2 - mu_2**2) / (2 * mu_1 - 2 * mu_2)
  print("intersect: x = ", intersect_x)

  plt.plot(x_1, y_1, label='P(x|w1)P(w1)', color='blue')
  plt.fill_between(x_1[x_1 < intersect_x], y_1[x_1 < intersect_x], color='blue', alpha=0.3)
  plt.plot(x_2, y_2, label='P(x|w2)P(w2)', color='orange')
  plt.fill_between(x_2[x_2 > intersect_x], y_2[x_2 > intersect_x], color='orange', alpha=0.3)
  plt.legend()
  plt.show()

vars = 2
mu_1 = 5
mu_2 = 0
p_w1 = 0.5
p_w2 = 0.5
plot_bayes_classifier(mu_1, mu_2, vars, vars, p_w1, p_w2)

### T3

In [ ]:
vars = 2
mu_1 = 5
mu_2 = 0
p_w1 = 0.75
p_w2 = 0.25
plot_bayes_classifier(mu_1, mu_2, vars, vars, p_w1, p_w2)

### OT2

As the image below

### OT3

Find new intersect_x

From P(x|w1) = P(x|w2)
1/sqrt(2*pi)/2*exp(-1/2*((x-4)^2)/2^2) = 1/sqrt(2*pi)/4*exp(-1/2*((x-0)^2)/4^2) 
x = -4 / 3 * (-4 + sqrt(4 + log(64)))

In [ ]:
vars1 = 2
vars2 = 4
mu_1 = 4
mu_2 = 0
p_w1 = 0.5
p_w2 = 0.5
intersect_x =  -4 / 3 * (-4 + math.sqrt(4 + math.log(64)))
plot_bayes_classifier(mu_1, mu_2, vars1, vars2, p_w1, p_w2, intersect_x=intersect_x)

## Employee Attrition Prediction


### read CSV

In [ ]:
df = pd.read_csv('hr-employee-attrition-with-null.csv')

### Dataset statistic

In [ ]:
df.describe()

In [ ]:
df.head()

### Feature transformation

In [ ]:
df.loc[df["Attrition"] == "no", "Attrition"] = 0.0
df.loc[df["Attrition"] == "yes", "Attrition"] = 1.0
string_categorical_col = ['Department', 'Attrition', 'BusinessTravel', 'EducationField', 'Gender', 'JobRole',
                              'MaritalStatus', 'Over18', 'OverTime']

# ENCODE STRING COLUMNS TO CATEGORICAL COLUMNS
for col in string_categorical_col:
    # INSERT CODE HERE
    df[col] = pd.Categorical(df[col]).codes

# HANDLE NULL NUMBERS
# INSERT CODE HERE
    

df = df.loc[:, ~df.columns.isin(['EmployeeNumber', 'Unnamed: 0', 'EmployeeCount', 'StandardHours', 'Over18'])]

###  Spliting data into train and test

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.1, stratify=df['Attrition'], random_state=7)

### Display histogram of each feature

In [ ]:
def display_histogram(df, feature, n_bin = 40):
        
        df_dropna = df[feature].dropna()
        plt.hist(df_dropna, n_bin)
        plt.xlabel(feature)
        plt.ylabel("Frequency")
        plt.show()
    # INSERT CODE HERE

### T4. Observe the histogram for Age, MonthlyIncome and DistanceFromHome. How many bins have zero counts? Do you think this is a good discretization? Why?

In [ ]:
def count_zero_bins(df, col_name, n_bins):
    df_drop_na = df[col_name].dropna()
    hist, bin__edge = np.histogram(df_drop_na, bins=n_bins)
    return np.count_nonzero(hist == 0)

features = ["Age", "MonthlyIncome", "DistanceFromHome"]
for feature in features:
    display_histogram(df, feature)
    print("Zero bin count of", feature, count_zero_bins(df, feature, n_bins=40))



From having no zero bins, the Age and MonthlyIncome features are good discretiztion.

While, the DistanceFromHome feature is not.

### T5. Can we use a Gaussian to estimate this histogram? Why? What about a Gaussian Mixture Model (GMM)?

Only the Age feature one can be estimated as Guassian since the rest don't look like a Normal Distribution.

The MonthlyIncome and DistanceFromHome features may be estimated using GMM or other distributions.

### T6. Now plot the histogram according to the method described above (with 10, 40, and 100 bins) and show 3 plots each for Age, MonthlyIncome, and DistanceFromHome. Which bin size is most sensible for each features? Why?

In [ ]:
bins = [10, 40, 100]

for bin in bins:
    for feature in features:
        display_histogram(df, feature, n_bin=bin)


For Age, the appropriate bins would be 10 because it can accurately describe the distribution of data. Not too rough and not too detailed.

For MonthlyIncome, the optimal bins are 40 because the data is highly distrubuted.

For DistanceFromHome, the appropriate bins are 10 because the DistanceFromHome data Will only be in a narrow range, no need to use many bins

### T7. For the rest of the features, which one should be discretized in order to be modeled by histograms? What are the criteria for choosing whether we should discretize a feature or not? Answer this and discretize those features into 10 bins each. In other words, figure out the bin edge for each feature, then use digitize() to convert the features to discrete values

In [ ]:
features = df.columns.to_list()
features.remove("Attrition")
for feature in features:
    if len(df[feature].unique()) < 10:
        features.remove(feature)
        continue
    display_histogram(df, feature, 10)

print(features)

### T8. What kind of distribution should we use to model histograms? (Answer a distribution name) What is the MLE for the likelihood distribution? (Describe how to do the MLE). Plot the likelihood distributions of MonthlyIncome, JobRole, HourlyRate, and MaritalStatus for different Attrition values.

In [ ]:
for col in ['MonthlyIncome', 'JobRole', 'HourlyRate', 'MaritalStatus']:
    plt.figure(figsize=(20, 5))
    leave_df = df.loc[df['Attrition'] == 1]
    stay_df = df.loc[df['Attrition'] == 0]

    display_histogram(leave_df, col, 10)

    display_histogram(stay_df, col, 10)



From the distribution, I chose Exponential Distribution.
p(x|\lambda) = \lambda * exp(-\lambda x)

The process to find MLE is
1. find the likelihood for each feature
2. find the derivative of sum of log of all likelihoods
3. set the derivative to zero and solve for the value that maximize the likelihoods


### T9. What is the prior distribution of the two classes?

In [ ]:
p_leave = df.loc[df['Attrition'] == 1, 'Attrition'].count() / df.shape[0]
p_stay = df.loc[df['Attrition'] == 0, 'Attrition'].count() /df.shape[0]

print("leave", p_leave)
print("stay", p_stay)

### T10. If we use the current Naive Bayes with our current Maximum Likelihood Estimates, we will find that some P (x i |attrition) will be zero and will result in the entire product term to be zero. Propose a method to fix this problem.

Use flooring for handling zeros by changing them to small numbers (e.g. 1e-20)

### T11. Implement your Naive Bayes classifier. Use the learned distributions to classify the test set. Don’t forget to allow your classifier to handle missing values in the test set. Report the overall Accuracy. Then, report the Precision, Recall, and F score for detecting attrition. See Lecture 1 for the definitions of each metric.


In [ ]:
from SimpleBayesClassifier import SimpleBayesClassifier

In [ ]:
data_train = df_train.to_numpy()
data_test = df_test.to_numpy()

In [252]:
test_size = 0.1

X = df_train.drop(columns=['Attrition']).to_numpy()
Y = df_train['Attrition'].to_numpy()

x_train = X[:-int(len(X) * test_size)]
y_train = Y[:-int(len(X) * test_size)]

x_test = X[-int(len(X) * test_size):]
y_test = Y[-int(len(X) * test_size):]

In [253]:
model = SimpleBayesClassifier(n_pos = (y_train==1).sum(), n_neg = (y_train==0).sum())

model.prior_pos, model.prior_neg

(0.15785054575986565, 0.8421494542401343)

In [254]:
def check_prior():
    """
    This function designed to test the implementation of the prior probability calculation in a Naive Bayes classifier. 
    Specifically, it checks if the classifier correctly computes the prior probabilities for the 
    negative and positive classes based on given input counts.
    """
    
    # prior_neg = 5/(5 + 5) = 0.5 and # prior_pos = 5/(5 + 5) = 0.5
    assert (SimpleBayesClassifier(5, 5).prior_pos, SimpleBayesClassifier(5, 5).prior_neg) == (0.5, 0.5)

    assert (SimpleBayesClassifier(3, 5).prior_pos, SimpleBayesClassifier(3, 5).prior_neg) == (0.375, 0.625)
    assert (SimpleBayesClassifier(0, 1).prior_pos, SimpleBayesClassifier(0, 1).prior_neg) == (0, 1)
    assert (SimpleBayesClassifier(1, 0).prior_pos, SimpleBayesClassifier(1, 0).prior_neg) == (1, 0)
    
check_prior()

In [255]:
model.fit_params(x_train, y_train)

([(array([0.02763819, 0.05653266, 0.15577889, 0.17211055, 0.17839196,
          0.15829146, 0.09547739, 0.07035176, 0.05150754, 0.0339196 ]),
   array([-inf, 22.2, 26.4, 30.6, 34.8, 39. , 43.2, 47.4, 51.6, 55.8,  inf])),
  (array([0.20338983, 0.        , 0.        , 0.08175474, 0.        ,
          0.        , 0.13758724, 0.        , 0.        , 0.5772682 ]),
   array([-inf, -0.7, -0.4, -0.1,  0.2,  0.5,  0.8,  1.1,  1.4,  1.7,  inf])),
  (array([0.11220715, 0.08138101, 0.0998767 , 0.10850801, 0.08138101,
          0.0974106 , 0.0838471 , 0.11960543, 0.11960543, 0.09617756]),
   array([  -inf,  244.4,  383.8,  523.2,  662.6,  802. ,  941.4, 1080.8,
          1220.2, 1359.6,    inf])),
  (array([0.19341974, 0.        , 0.        , 0.03788634, 0.        ,
          0.        , 0.54735793, 0.        , 0.        , 0.22133599]),
   array([-inf, -0.7, -0.4, -0.1,  0.2,  0.5,  0.8,  1.1,  1.4,  1.7,  inf])),
  (array([0.35903919, 0.13400759, 0.16687737, 0.09987358, 0.02275601,
          0.04

In [256]:
def check_fit_params():

    """
    This function is designed to test the fit_params method of a SimpleBayesClassifier. 
    This method is presumably responsible for computing parameters for a Naive Bayes classifier 
    based on the provided training data. The parameters in this context is bins and edges from each histogram.
    """

    T = SimpleBayesClassifier(2, 2)
    X_TRAIN_CASE_1 = np.array([
        [0, 1, 2, 3],
        [1, 2, 3, 4],
        [2, 3, 4, 5],
        [3, 4, 5, 6]
    ])
    Y_TRAIN_CASE_1 = np.array([0, 1, 0, 1])
    STAY_PARAMS_1, LEAVE_PARAMS_1 = T.fit_params(X_TRAIN_CASE_1, Y_TRAIN_CASE_1)

    print("STAY PARAMETERS")
    for f_idx in range(len(STAY_PARAMS_1)):
        print(f"Feature : {f_idx}")
        print(f"BINS : {STAY_PARAMS_1[f_idx][0]}")
        print(f"EDGES : {STAY_PARAMS_1[f_idx][1]}")
    print("")    
    print("LEAVE PARAMETERS")
    for f_idx in range(len(STAY_PARAMS_1)):
        print(f"Feature : {f_idx}")
        print(f"BINS : {LEAVE_PARAMS_1[f_idx][0]}")
        print(f"EDGES : {LEAVE_PARAMS_1[f_idx][1]}")

check_fit_params()

STAY PARAMETERS
Feature : 0
BINS : [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5]
EDGES : [-inf  0.2  0.4  0.6  0.8  1.   1.2  1.4  1.6  1.8  inf]
Feature : 1
BINS : [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5]
EDGES : [-inf  1.2  1.4  1.6  1.8  2.   2.2  2.4  2.6  2.8  inf]
Feature : 2
BINS : [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5]
EDGES : [-inf  2.2  2.4  2.6  2.8  3.   3.2  3.4  3.6  3.8  inf]
Feature : 3
BINS : [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5]
EDGES : [-inf  3.2  3.4  3.6  3.8  4.   4.2  4.4  4.6  4.8  inf]

LEAVE PARAMETERS
Feature : 0
BINS : [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5]
EDGES : [-inf  1.2  1.4  1.6  1.8  2.   2.2  2.4  2.6  2.8  inf]
Feature : 1
BINS : [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5]
EDGES : [-inf  2.2  2.4  2.6  2.8  3.   3.2  3.4  3.6  3.8  inf]
Feature : 2
BINS : [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5]
EDGES : [-inf  3.2  3.4  3.6  3.8  4.   4.2  4.4  4.6  4.8  inf]
Feature : 3
BINS : [0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.5]
EDGES : [-inf  4.2  4.4

In [257]:
y_pred = model.predict(x = x_test)

IndexError: index 10 is out of bounds for axis 0 with size 10

In [ ]:
def evaluate(y_true, y_pred, show_result = True):

  return accuracy, precision, recall, F1, fpr

In [ ]:
evaluate(y_test, y_pred)

### T12. Use the learned distributions to classify the test set. Report the results using the same metric as the previous question.


In [ ]:
model.fit_gaussian_params(x_train, y_train)

In [ ]:
def check_fit_gaussian_params():

    """
    This function is designed to test the fit_gaussian_params method of a SimpleBayesClassifier. 
    This method is presumably responsible for computing parameters for a Naive Bayes classifier 
    based on the provided training data. The parameters in this context is mean and STD.
    """

    T = SimpleBayesClassifier(2, 2)
    X_TRAIN_CASE_1 = np.array([
        [0, 1, 2, 3],
        [1, 2, 3, 4],
        [2, 3, 4, 5],
        [3, 4, 5, 6]
    ])
    Y_TRAIN_CASE_1 = np.array([0, 1, 0, 1])
    STAY_PARAMS_1, LEAVE_PARAMS_1 = T.fit_gaussian_params(X_TRAIN_CASE_1, Y_TRAIN_CASE_1)

    print("STAY PARAMETERS")
    for f_idx in range(len(STAY_PARAMS_1)):
        print(f"Feature : {f_idx}")
        print(f"Mean : {STAY_PARAMS_1[f_idx][0]}")
        print(f"STD. : {STAY_PARAMS_1[f_idx][1]}")
    print("")    
    print("LEAVE PARAMETERS")
    for f_idx in range(len(STAY_PARAMS_1)):
        print(f"Feature : {f_idx}")
        print(f"Mean : {LEAVE_PARAMS_1[f_idx][0]}")
        print(f"STD. : {LEAVE_PARAMS_1[f_idx][1]}")
    
check_fit_gaussian_params()

In [ ]:
y_pred = model.gaussian_predict(x_test)

In [ ]:
evaluate(y_test, y_pred)

### T13 : The random choice baseline is the accuracy if you make a random guess for each test sample. Give random guess (50% leaving, and 50% staying) to the test samples. Report the overall Accuracy. Then, report the Precision, Recall, and F score for attrition prediction using the random choice baseline.

### T14. The majority rule is the accuracy if you use the most frequent class from the training set as the classification decision. Report the overall Accuracy. Then, report the Precision, Recall, and F score for attrition prediction using the majority rule baseline.



### T15. Compare the two baselines with your Naive Bayes classifier.


### T16. Use the following threshold values
$ t = np.arange(-5,5,0.05) $
### find the best accuracy, and F score (and the corresponding thresholds)


### T17. Plot the RoC of your classifier.

### T18. Change the number of discretization bins to 5. What happens to the RoC curve? Which discretization is better? The number of discretization bins can be considered as a hyperparameter, and must be chosen by comparing the final performance.
